## 4. Создание и преобразование столбцов с помощью функций

In [22]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',') 
melb_df = melb_data.copy()

✍ Библиотека Pandas предоставляет большое количество возможностей для преобразований данных, однако иногда необходимо совершать более сложные манипуляции над столбцами. Например, из столбцов, содержащих в себе некоторый текст, необходимо специальным образом извлечь определённые слова, даты или числа.

→ Для таких случаев Pandas не имеет специальных методов, однако позволяет расширить свою функциональность за счёт использования пользовательских функций. 

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода **apply()**. В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода **nunique()**:

In [23]:
print(melb_df['Address'].nunique())

13378


Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет **статистической значимости**, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

##### Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

In [ ]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию **get_street_type** в аргумент метода столбца **apply()**. В результате получим объект Series, который положим в переменную **street_types**:

In [ ]:
street_types = melb_df['Address'].apply(get_street_type)
street_types

Обратите внимание, что функция пишется для одного элемента столбца, а метод **apply()** применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

In [ ]:
street_types.nunique()

У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода **value_counts**:

In [31]:
street_types.value_counts()

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как *'other'* (другие).

Для этого к результату метода **value_counts** применим метод **nlargest()**, который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута **index**, а результат занесём в переменную popular_stypes:

In [26]:
popular_stypes = street_types.value_counts().nlargest(10).index
popular_stypes

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём **lambda-функцию**, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [27]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType']

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

Посмотрим на результирующее число уникальных подтипов:

In [30]:
melb_df['StreetType'].value_counts()

St       8012
Rd       2825
Ct        612
Dr        447
other     420
Av        321
Gr        311
Pde       211
Pl        169
Cr        152
Cl        100
Name: StreetType, dtype: int64

Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [ ]:
melb_df = melb_df.drop('Address', axis=1)

Таким образом, с помощью написания собственных функций и их комбинирования с методом **apply()** из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы. 

**Примечание.** Внимательный читатель наверняка обратит внимание на то, что мы допустили небольшую ошибку!

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

Мы предлагаем вам самостоятельно разобраться с этой ошибкой: попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.

→ Обратите внимание, что данное преобразование необходимо применить до сокращения количества уникальных категорий.

In [32]:
melb_df['Street'] = melb_df['Address'].apply(get_street_type)
def change_street(address):
    street = {'Avenue':'Av', 'Boulevard':'Bvd', 'Parade':'Pde'}
    if address in ['Avenue', 'Boulevard', 'Parade']:
        address = street[address]
    else:
        address
    return address
melb_df['Street_mod'] = melb_df['Street'].apply(change_street)
sorted(melb_df['Street_mod'].unique())

['Athol',
 'Av',
 'Ave',
 'Bvd',
 'Cct',
 'Cir',
 'Cl',
 'Corso',
 'Cove',
 'Cr',
 'Crescent',
 'Crofts',
 'Crossway',
 'Ct',
 'Dell',
 'Dr',
 'East',
 'Esp',
 'Esplanade',
 'Eyrie',
 'Fairway',
 'Gdns',
 'Glade',
 'Gr',
 'Gra',
 'Grand',
 'Grange',
 'Grn',
 'Grove',
 'Highway',
 'Hts',
 'Hwy',
 'La',
 'Loop',
 'Mews',
 'Nook',
 'Outlook',
 'Pde',
 'Pl',
 'Qy',
 'Rd',
 'Res',
 'Ridge',
 'Righi',
 'Sq',
 'St',
 'Strand',
 'Summit',
 'Tce',
 'Terrace',
 'Victoria',
 'Woodland',
 'Wy']

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1. Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.
2. Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).
3. Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.
4. Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.
5. Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

In [36]:
seller = melb_df['SellerG'].value_counts().nlargest(49).index
seller

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object')

In [37]:
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in seller else 'other')
melb_df['SellerG'].value_counts()

Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Eview              51
Purplebricks       51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Nick               40
Alexkarbon

In [38]:
a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

1.3


Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.

Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.

Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции. Кроме того, учтите возможные вариации слов месяц (месяца, месяцев) и год (года, лет).

Примените вашу функцию к Series experience_col с помощью метода apply().

In [ ]:
import pandas as pd

def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
            
    return (int(year)*12) + int(month)

if __name__ == '__main__':
    experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
    experience_month = experience_col.apply(get_experience)
    print(experience_month)